# Section 1
## Read in database credentials
Below the credentials for connecting to the database are read into variables by extracting the lines from the local file. The local file is not included in the repo.

In [3]:
db_name = ""
db_user = ""
db_pass = ""
db_host = ""
with open("database_credentials.txt") as f:
    db_name = f.readline().strip()
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_host = f.readline().strip()

## Test connection
The next code segment tests to ensure that the database connection is working properly.

In [4]:
import pymysql as pms

In [5]:
try:
    con = pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    print("Successfully connected")
finally:
    if con:
        print("Closing connection")
        con.close()

Successfully connected
Closing connection


## Making a table
The below code creates a sample table.

In [6]:
try:
    con = pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    
    cur = con.cursor()
    #SQL used for table creation
    sql = """
    CREATE TABLE IF NOT EXISTS dept(
        deptno INT(2),
        dname VARCHAR(14),
        loc VARCHAR(13),
        CONSTRAINT pk_dept PRIMARY KEY (deptno)
    );
    """
    #Execute
    cur.execute(sql)
    
    sql = """
    CREATE TABLE IF NOT EXISTS emp(
        empno INT(4),
        ename VARCHAR(10),
        job VARCHAR(9),
        mgr INT(4),
        hiredate DATE,
        sal FLOAT(7,2),
        comm FLOAT(7,2),
        deptno INT(2),
        CONSTRAINT pk_emp PRIMARY KEY (empno),
        CONSTRAINT fk_dpetno FOREIGN KEY (deptno) REFERENCES DEPT (deptno)
    );
    """
    cur.execute(sql)
finally:
    if con:
        cur.close()
        con.close()

## Insertions
Below some Oracle code is translated into MySQL code. This code is responsible for populating the databases with entries.

In [7]:
try:
    con = pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name)
    #Use to save inserts
    con.autocommit(True)
    
    cur = con.cursor()
    
    #Skip if database is already populated
    cur.execute("select * from dept")
    if not cur.fetchone():
        #Oracle statements
        oracle = """
            insert into dept
            values(10, 'ACCOUNTING', 'NEW YORK');
            insert into dept
            values(20, 'RESEARCH', 'DALLAS');
            insert into dept
            values(30, 'SALES', 'CHICAGO');
            insert into dept
            values(40, 'OPERATIONS', 'BOSTON');

            insert into emp
            values(
             7839, 'KING', 'PRESIDENT', null,
             to_date('17-11-1981','dd-mm-yyyy'),
             5000, null, 10
            );
            insert into emp
            values(
             7698, 'BLAKE', 'MANAGER', 7839,
             to_date('1-5-1981','dd-mm-yyyy'),
             2850, null, 30
            );
            insert into emp
            values(
             7782, 'CLARK', 'MANAGER', 7839,
             to_date('9-6-1981','dd-mm-yyyy'),
             2450, null, 10
            );
            insert into emp
            values(
             7566, 'JONES', 'MANAGER', 7839,
             to_date('2-4-1981','dd-mm-yyyy'),
             2975, null, 20
            );
            insert into emp
            values(
             7788, 'SCOTT', 'ANALYST', 7566,
             to_date('9-12-1981','dd-mm-yyyy'),
             3000, null, 20
            );
            insert into emp
            values(
             7902, 'FORD', 'ANALYST', 7566,
             to_date('3-12-1981','dd-mm-yyyy'),
             3000, null, 20
            );
            insert into emp
            values(
             7369, 'SMITH', 'CLERK', 7902,
             to_date('17-12-1980','dd-mm-yyyy'),
             800, null, 20
            );
            insert into emp
            values(
             7499, 'ALLEN', 'SALESMAN', 7698,
             to_date('20-2-1981','dd-mm-yyyy'),
             1600, 300, 30
            );
            insert into emp
            values(
             7521, 'WARD', 'SALESMAN', 7698,
             to_date('22-2-1981','dd-mm-yyyy'),
             1250, 500, 30
            );
            insert into emp
            values(
             7654, 'MARTIN', 'SALESMAN', 7698,
             to_date('28-9-1981','dd-mm-yyyy'),
             1250, 1400, 30
            );
            insert into emp
            values(
             7844, 'TURNER', 'SALESMAN', 7698,
             to_date('8-9-1981','dd-mm-yyyy'),
             1500, 0, 30
            );
            insert into emp
            values(
             7876, 'ADAMS', 'CLERK', 7788,
             to_date('12-1-1983', 'dd-mm-yyyy'),
             1100, null, 20
            );
            insert into emp
            values(
             7900, 'JAMES', 'CLERK', 7698,
             to_date('3-12-1981','dd-mm-yyyy'),
             950, null, 30
            );
            insert into emp
            values(
             7934, 'MILLER', 'CLERK', 7782,
             to_date('23-1-1982','dd-mm-yyyy'),
             1300, null, 10
            );
        """

        #MySQL statements
        mysql = oracle.split(";")
        #Map MySQL syntax
        mysql1 = list(map(lambda s : s.replace("dept", "dept (deptno, dname, loc)"), mysql[:4]))
        mysql2 = list(
            map(
                lambda s : s.replace(
                    "emp", "emp (empno, ename, job, mgr, hiredate, sal, comm, deptno)").replace(
                    "to_date", "str_to_date").replace(
                    "dd-mm-yyyy", "%d-%m-%Y"),
                mysql[4:]))
        #Rejoin query string
        mysql = ";".join(mysql1 + mysql2).split(";")[:-1]
        #Execute each statement individually
        for s in mysql:
            cur.execute(s)
finally:
    if con:
        con.close()

## Shorthand connect to database
This method will just return a new database connection with the default credentials made available above.

In [8]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## DISTINCT select
Selecting the distinct values associated with a specific column is demonstrated below.

In [9]:
try:
    con = get_connect()
    cur = con.cursor()
    
    #Distinct selection
    sql = """
        SELECT DISTINCT job FROM emp
    """
    cur.execute(sql)
    print(cur.fetchall())
finally:
    if con:
        con.close()

(('CLERK',), ('SALESMAN',), ('MANAGER',), ('ANALYST',), ('PRESIDENT',))
